In [1]:
import tensorflow as tf
import numpy as np

In [16]:
labels = 10
labelEmbedDimension=5
num_filters=3
m=15
window_width=2
labelsPlaceholder = tf.placeholder(tf.float32,shape=[labels,labels])
target = tf.placeholder(tf.float32,shape=[labels])

In [17]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))

def new_biases(length):
    return tf.Variable(tf.constant(0.05, shape=[length]))

In [18]:
def getLabelEmbeddings(labelsPlaceholder,labelsCount,labelEmdeddingLength):
    embeddings = tf.Variable(tf.random_uniform([labelsCount, labelEmdeddingLength], -1.0, 1.0))
    rel = tf.matmul(labelsPlaceholder,embeddings)
    return rel,embeddings
relMatrix,embedMatrix=getLabelEmbeddings(labelsPlaceholder,labels,labelEmbedDimension)

In [19]:
def convLayer(sentenceMatrix,relationMatrix,window_width,num_filters,shape):
    weights = new_weights(shape=shape)
    biases = new_biases(length=num_filters)    
    inputMatrix = tf.matmul(sentenceMatrix,tf.transpose(relationMatrix))
    inputMatrix = tf.reshape(inputMatrix,shape=[1,m,labels,1])
    convOutput = tf.nn.conv2d(input=inputMatrix,filter=weights,strides=[1,1,1,1],padding='SAME')
    convOutput = tf.nn.tanh(convOutput)    
    return convOutput

def fullyConnectedLayer(bagRepresenation,num_inputs,num_outputs):
    weights = new_weights(shape=[num_inputs, num_outputs])
    biases = new_biases(length=num_outputs)
    bagRepresenation=tf.contrib.layers.flatten(bagRepresenation)
    layer = tf.nn.sigmoid(tf.matmul(bagRepresenation, weights) + biases)
    return layer,weights


In [20]:
sentenceMatrix = np.random.rand(m,labelEmbedDimension).astype(np.float32)
shape = [1,4,1,num_filters]
inputSize = labels*m*num_filters

In [21]:
convOutput = convLayer(sentenceMatrix,relMatrix,window_width,num_filters,shape)
prediction,weights_fc=fullyConnectedLayer(convOutput,inputSize,labels)
cross_entropy = -tf.reduce_sum(((target*tf.log(prediction + 1e-9)) + ((1-target) * tf.log(1 - prediction + 1e-9)) )  , name='xentropy' ) 
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

In [22]:
session = tf.Session()
session.run(tf.global_variables_initializer())
feed_dict_input={}
entityLabel = np.array([[0]*labels]*labels)
entityLabel[0][0]=1
entityLabel[2][2]=1
# print(entityLabel)
feed_dict_input[labelsPlaceholder]=entityLabel

out=session.run(prediction,feed_dict=feed_dict_input)

In [15]:
print(entityLabel.shape)

(10, 10)


(1, 10)